In [3]:
!pip install tensorflow pandas numpy scikit-learn folium geopy ipython

In [5]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import folium
from geopy.geocoders import Nominatim
from IPython.display import display

# Load dataset
df = pd.read_csv("Crime_Dataset.csv")

# Extract features and target variables
X = df[['Region']]  # Region as input
y = df.drop(columns=['Region'])  # Crime categories as target

# One-hot encode the Region column
encoder = OneHotEncoder(sparse_output=False)
X_encoded = encoder.fit_transform(X)

# Normalize the crime data
scaler = StandardScaler()
y_scaled = scaler.fit_transform(y)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y_scaled, test_size=0.2, random_state=42)

# Build the ANN model
model = keras.Sequential([
    keras.layers.Input(shape=(X_train.shape[1],)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(y_train.shape[1], activation='sigmoid')  # Output layer for crime probabilities
])

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=8, validation_data=(X_test, y_test), verbose=1)

# Save the trained model
model.save("crime_prediction_model.h5")

Epoch 1/50
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.9379 - mae: 0.5131 - val_loss: 0.0444 - val_mae: 0.1256
Epoch 2/50
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.8014 - mae: 0.2092 - val_loss: 0.0366 - val_mae: 0.0887
Epoch 3/50
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.4757 - mae: 0.1376 - val_loss: 0.0374 - val_mae: 0.0941
Epoch 4/50
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5044 - mae: 0.1013 - val_loss: 0.0406 - val_mae: 0.1126
Epoch 5/50
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5751 - mae: 0.1154 - val_loss: 0.0413 - val_mae: 0.1160
Epoch 6/50
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5570 - mae: 0.1066 - val_loss: 0.0396 - val_mae: 0.1074
Epoch 7/50
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1820 - mae: 0.0916 - val_loss: 0.0376 - val_mae: 0.0956
Epoch 8/50
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.9050 - mae: 0.1491 - val_loss: 0.0371 - val_mae: 0.0920
Epoch 9/50
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3966 - mae: 

In [7]:
precautions_dict = {
    "Abandoning Infant": ["Report abandoned infants to authorities.", "Ensure child safety measures.", "Support child welfare programs."],
    "Abetment of Suicide": ["Encourage mental health support.", "Report coercion or threats.", "Promote awareness about suicide prevention."],
    "Acid Attack": ["Be aware of surroundings.", "Avoid conflicts with hostile individuals.", "Report any threats immediately."],
    "Arson": ["Ensure fire safety measures.", "Report suspicious activities.", "Avoid storing flammable substances openly."],
    "Assault on Women with intent to outrage her Modesty": ["Avoid isolated locations.", "Use personal safety apps.", "Report incidents immediately."],
    "Attempt to Acid Attack": ["Stay vigilant in public spaces.", "Report threats to authorities.", "Educate about acid attack laws."],
    "Attempt to commit Culpable Homicide": ["Stay away from hostile situations.", "Alert security personnel if threatened.", "Avoid interactions with violent individuals."],
    "Attempt to commit Murder": ["Avoid suspicious individuals.", "Report threats to authorities.", "Stay in secure locations."],
    "Attempt to commit Rape": ["Avoid secluded areas.", "Use emergency contacts.", "Report harassment immediately."],
    "Attempt to commit Suicide": ["Seek mental health assistance.", "Encourage counseling services.", "Provide emotional support."],
    "Auto Theft": ["Use anti-theft devices.", "Park in secure locations.", "Avoid leaving valuables inside vehicles."],
    "Buying or Selling of Minors for prostitution": ["Report human trafficking activities.", "Educate about child protection laws.", "Support anti-trafficking initiatives."],
    "Causing hurt by rash driving": ["Follow traffic rules.", "Avoid overspeeding.", "Stay alert while driving."],
    "Cheating": ["Verify before making financial transactions.", "Avoid sharing personal details.", "Be skeptical of unrealistic offers."],
    "Child Abuse": ["Report suspected child abuse immediately.", "Educate children about personal safety.", "Support child protection agencies."],
    "Child Marriage": ["Report underage marriages.", "Promote awareness of legal marriage age.", "Support victims of forced marriages."],
    "Child Pornography": ["Monitor children's internet usage.", "Report online child exploitation.", "Educate about online safety."],
    "Counterfeiting": ["Verify authenticity of currency and goods.", "Report counterfeit items.", "Avoid suspicious transactions."],
    "Cyber Stalking": ["Do not share personal information online.", "Block and report suspicious users.", "Enable privacy settings on social media."],
    "Identity Theft": ["Use multi-factor authentication.", "Monitor bank statements regularly.", "Report unauthorized transactions."],
    "Hate Crimes": ["Report hate speech and violence.", "Support victims of hate crimes.", "Promote community harmony."],
    "Bribery": ["Refuse to participate in bribery.", "Report corruption cases.", "Support ethical governance."],
    "Human Trafficking": ["Report suspicious activities.", "Avoid illegal employment agencies.", "Support anti-trafficking organizations."],
    "Extortion": ["Do not respond to blackmail threats.", "Report extortion attempts to authorities.", "Secure important personal and business data."],
    "Insult to Modesty of Women": ["Call out inappropriate behavior.", "Use legal channels for complaints.", "Support awareness campaigns."],
    "Forced Labor": ["Report exploitation of workers.", "Encourage fair labor policies.", "Support ethical businesses."],
    "Public Nuisance": ["Respect public spaces.", "Report disturbances to authorities.", "Encourage civic responsibility."],
    "Sexual Exploitation": ["Educate about consent and rights.", "Support victims with legal aid.", "Report perpetrators to authorities."],
    "Theft": ["Keep valuables secure.", "Avoid displaying expensive items in public.", "Report theft immediately."],
    "Terrorism": ["Stay aware of surroundings.", "Report suspicious activities.", "Follow emergency protocols."],
    "Vandalism": ["Secure property with surveillance.", "Report acts of vandalism.", "Promote community awareness."],
    "Voyeurism": ["Report hidden cameras in public spaces.", "Be cautious in private settings.", "Support strict privacy laws."],
    "Misuse of Social Media": ["Verify information before sharing.", "Report online abuse and misinformation.", "Use responsible social media practices."]
}

In [9]:
def get_coordinates(region):
    geolocator = Nominatim(user_agent="crime_map")
    location = geolocator.geocode(region)
    if location:
        return (location.latitude, location.longitude)
    return None

def visualize_crime(region_name):
    region_encoded = encoder.transform(pd.DataFrame([[region_name]], columns=['Region']))
    predictions = model.predict(region_encoded)
    predicted_crimes = scaler.inverse_transform(predictions)  # Convert back to original scale
    crime_data = pd.DataFrame(predicted_crimes, columns=y.columns)
    
    # Select top 10 crimes with highest probability
    top_crimes = crime_data.iloc[0].nlargest(10)
    
    # Fetch precautions for top crimes
    precautions_output = {}
    for crime in top_crimes.index:
        precautions_output[crime] = precautions_dict.get(crime, ["No specific precautions available."])

    coordinates = get_coordinates(region_name)
    if not coordinates:
        print("Location not found on the map!")
        return
    
    crime_map = folium.Map(location=coordinates, zoom_start=10)
    
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'yellow', 'pink', 'brown', 'gray', 'black']
    legend_html = '<div style="position: fixed; bottom: 50px; left: 50px; width: 250px; height: auto; background-color: white; z-index:9999; padding: 10px; border-radius: 5px;">'
    legend_html += '<h4>Crime Legend</h4>'
    
    for i, (crime, severity) in enumerate(top_crimes.items()):
        offset_lat = coordinates[0] + (0.01 * (i+1))
        offset_lon = coordinates[1] + (0.01 * (i+1))
        
        folium.CircleMarker(
            location=[offset_lat, offset_lon],
            radius=7,
            color=colors[i % len(colors)],
            fill=True,
            fill_color=colors[i % len(colors)],
            fill_opacity=0.6,
            popup=f"{crime}: {severity:.2f} (Severity Level)"
        ).add_to(crime_map)
        
        legend_html += f'<p style="margin:0;"><span style="display:inline-block; width:10px; height:10px; background-color:{colors[i % len(colors)]}; margin-right:5px;"></span>{crime}</p>'
    
    legend_html += '</div>'
    crime_map.get_root().html.add_child(folium.Element(legend_html))
    
    display(crime_map)

    # Display precautions
    print("\nPrecautions for Predicted Crimes:\n")
    for crime, precautions in precautions_output.items():
        print(f"{crime}:")
        for precaution in precautions:
            print(f"- {precaution}")
        print()

# Example usage
visualize_crime("Hyderabad city, India")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step



Precautions for Predicted Crimes:

Other IPC crimes:
- No specific precautions available.

Theft:
- Keep valuables secure.
- Avoid displaying expensive items in public.
- Report theft immediately.

Incidence of Rash Driving:
- No specific precautions available.

Other Thefts:
- No specific precautions available.

Auto Theft:
- Use anti-theft devices.
- Park in secure locations.
- Avoid leaving valuables inside vehicles.

Causing Death by Negligence:
- No specific precautions available.

Cruelty by Husband or his Relatives:
- No specific precautions available.

Deaths due to negligent driving/act:
- No specific precautions available.

Criminal Trespass/Burglary:
- No specific precautions available.

Cheating:
- Verify before making financial transactions.
- Avoid sharing personal details.
- Be skeptical of unrealistic offers.

